# Coarse solver example

This example solves the multi-scale cos problem by pretraining a single network over the full domain before fixing its weights and adding it effectively as another network in the FBPINN ansatz summation whilst training the FBPINN.

In [ ]:
import sys

import numpy as np

import problems
from active_schedulers import _ActiveScheduler
from constants import Constants, get_subdomain_xs, get_subdomain_ws
from main import FBPINNTrainer, PINNTrainer
from trainersBase import train_models_multiprocess

In [ ]:
# define custom active scheduler

class CoarseStartSchedulerND(_ActiveScheduler):
    "All models are active all of the time"
    
    name = "All"
    
    def __iter__(self):
        
        n_pretrain = 3000
        assert n_pretrain <= self.N_STEPS
        
        # First, pre-train coarse model
        for i in range(n_pretrain):
            if i == 0: yield np.zeros(self.nm, dtype=int), 1# 1 means coarse model is "active"
            else: yield None
        
        # Then, set coarse model to fixed and train subdomain networks
        for i in range(self.N_STEPS-n_pretrain):
            if i == 0: yield np.ones(self.nm, dtype=int), 2# 2 means coarse model is "fixed"
            else: yield None
                
                

In [ ]:
# Cos multi w1=1 w2=15
P = problems.Cos_multi1D_1(w1=1, w2=15, A=0)
subdomain_xs = get_subdomain_xs([np.array([2,]*30)], [2*np.pi])
boundary_n = (1/P.w2,)
y_n = (0,2)
batch_size = (3000,)
batch_size_test = (5000,)
n_steps = 5000
n_hidden, n_layers = 16, 2
width = 0.7
subdomain_ws = get_subdomain_ws(subdomain_xs, width)
A = CoarseStartSchedulerND
args = ()
plot_lims = (1.1, False)
random = False

sampler = "r" if random else "m"
c = Constants(
              RUN="final_FBPINN_%s_%sh_%sl_%sb_%s_%sw_%s"%(P.name, n_hidden, n_layers, batch_size[0], sampler, width, A.name),
              P=P,
              SUBDOMAIN_XS=subdomain_xs,
              SUBDOMAIN_WS=subdomain_ws,
              BOUNDARY_N=boundary_n,
              Y_N=y_n,
              ACTIVE_SCHEDULER=A,
              ACTIVE_SCHEDULER_ARGS=args,
              N_HIDDEN=n_hidden,
              N_LAYERS=n_layers,
              BATCH_SIZE=batch_size,
              RANDOM=random,
              N_STEPS=n_steps,
              BATCH_SIZE_TEST=batch_size_test,
              PLOT_LIMS=plot_lims,
              TEST_FREQ=1000,
              SHOW_FIGURES=True,
              CLEAR_OUTPUT=True,
              SAVE_FIGURES=True,
              )

run = FBPINNTrainer(c)
run.train()